<a href="https://colab.research.google.com/github/ccheitor/IntroPySpark/blob/main/nbk_intro_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Etapa 0 - Preparação
    - Bloco reservado para instalação do pyspark no ambiente.
    - Se já tiver instalado, pode avançar esta parte!

In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=3f1834b77b299c5435d2bae7fdfbe76e59759125bab524d5f5c820971b3fec22
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Etapa 1 - Iniciar sessão do Spark

In [4]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName(" Execução do trabalho Interativo")\
  .getOrCreate()

spark.version

'3.5.0'

## Etapa 2 - Carregar os arquivos

In [5]:
df_titles = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/Spark/title_basics.tsv', header=True,
inferSchema=True, sep='\t')

df_ratings = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/Spark/title_ratings.tsv', header=True,
inferSchema=True, sep='\t')

In [6]:
df_titles.printSchema()
df_titles.show()

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,

In [7]:
df_ratings.printSchema()
df_ratings.show()

root
 |-- tconst: string (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: integer (nullable = true)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    1809|
|tt0000002|          6.0|     233|
|tt0000003|          6.5|    1560|
|tt0000004|          6.1|     152|
|tt0000005|          6.2|    2383|
|tt0000006|          5.1|     157|
|tt0000007|          5.4|     746|
|tt0000008|          5.5|    1965|
|tt0000009|          5.8|     189|
|tt0000010|          6.9|    6530|
|tt0000011|          5.2|     323|
|tt0000012|          7.4|   11188|
|tt0000013|          5.8|    1726|
|tt0000014|          7.1|    5023|
|tt0000015|          6.2|     947|
|tt0000016|          5.9|    1325|
|tt0000017|          4.6|     292|
|tt0000018|          5.3|     537|
|tt0000019|          5.3|      28|
|tt0000020|          5.0|     315|
+---------+-------------+--------+
only showing top 20 rows



## Etapa 3 - Levantamento dos dados

In [8]:
# Questão 1
# Quantos filmes (incluindo os da televisão) foram lançados no ano de 2015?
df_titles.filter(df_titles.titleType.isin(['movie', 'tvMovie']) & (df_titles.startYear == 2015)).count()

19987

In [12]:
# Questão 2
# Qual o gênero de títulos mais frequente?

from pyspark.sql.functions import desc
df_titles.groupBy("genres").count().orderBy(desc("count")).first()

Row(genres='Drama', count=880649)

In [10]:
# Relacionamento das tabelas
df_joined = df_titles.join(df_ratings, on='tconst', how='inner')

In [11]:
# Questão 3
# Qual o gênero com a melhor nota média de títulos?

# Registre o DataFrame df_joined como uma tabela temporária
df_joined.createOrReplaceTempView("joined")

# Execute a consulta SQL para calcular a média das classificações por gênero
best_genre = spark.sql("""
    SELECT genres, AVG(averageRating) as avg_rating
    FROM joined
    GROUP BY genres
    ORDER BY avg_rating DESC
    LIMIT 1
""")

# Exiba o gênero com a melhor nota média
best_genre.show()

+--------------------+----------+
|              genres|avg_rating|
+--------------------+----------+
|Comedy,History,We...|       9.8|
+--------------------+----------+



In [13]:
# Questão 4
# Qual o vídeo game do gênero aventura mais bem avaliado em 2020?

spark.sql("""
    SELECT primaryTitle, genres, averageRating
    FROM joined
    WHERE titleType = "videoGame"
    AND startYear = "2020"
    AND genres LIKE "%Adventure%"
    ORDER BY averageRating DESC
    LIMIT 1
""").show()

+---------------+--------------------+-------------+
|   primaryTitle|              genres|averageRating|
+---------------+--------------------+-------------+
|Half-Life: Alyx|Action,Adventure,...|          9.5|
+---------------+--------------------+-------------+



In [14]:
# Questão 5
# Quantos títulos de filmes diferentes existem?

df_titles.select('primaryTitle').distinct().count()

3931670

In [15]:
# Questão 6
# Qual a duração média dos filmes com conteúdo adulto?

df_titles.filter(df_titles.isAdult == 1).describe().show()

+-------+---------+---------+--------------------+--------------------+-------+------------------+------------------+-----------------+------+
|summary|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|         startYear|           endYear|   runtimeMinutes|genres|
+-------+---------+---------+--------------------+--------------------+-------+------------------+------------------+-----------------+------+
|  count|   250127|   250127|              250127|              250127| 250127|            250127|            250127|           250127|250127|
|   mean|     NULL|     NULL|            Infinity|            Infinity|    1.0|2009.9655151651386|2013.4869358669835|92.79938555059914|  NULL|
| stddev|     NULL|     NULL|                 NaN|                 NaN|    0.0| 9.116491116779232| 6.198536337845521|57.18982244754778|  NULL|
|    min|tt0060313|    movie|"A Different Side...|"A Different Side...|      1|              1901|              1980|                1|Action|

In [16]:
# Questão 7
# Quantos filmes têm o título atual (“primary”) diferente do título original?

df_titles.filter(df_titles.primaryTitle != df_titles.originalTitle).count()

125056

In [18]:
# Questão 8
# Qual o filme que tem o nome mais longo?

from pyspark.sql.functions import length,col

df_titles.withColumn('titleLength', length(col('primaryTitle'))) \
                           .orderBy(length(col('primaryTitle')), ascending=False) \
                           .limit(1).show()

+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+------+-----------+
|    tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|genres|titleLength|
+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+------+-----------+
|tt12985206|    video|Otoko wa chi _ ko...|Otoko wa chi _ ko...|      1|     2020|     \N|            99| Adult|        419|
+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+------+-----------+



In [19]:
# Questão 9
# Qual filme tem a maior quantidade de votos?

df_ratings.orderBy('numVotes', ascending=False).limit(1).show()

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0111161|          9.3| 2449517|
+---------+-------------+--------+



In [20]:
# Questão 10
# Qual é a menor nota média de um filme?

df_ratings.orderBy('averageRating', ascending=True).limit(1)

DataFrame[tconst: string, averageRating: double, numVotes: int]